<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-loader" data-toc-modified-id="Data-loader-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data loader</a></span></li></ul></div>

# Подключение библиотек

In [1]:
import pandas as pd
import json
import torch
import os

from skimage import io, transform
import matplotlib.pyplot as plt

import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18

from PIL import Image
import numpy as np

/usr/local/lib/python3.8/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


## Data loader

In [2]:
class PlanesDataset(Dataset):
    
    def __init__(self, annotation_file, root_dir, transform=None):
        
        with open('/home/alex/Рабочий стол/python/yandex/planes/labels.json') as json_file:
            self.annotation = json.load(json_file)
            
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotation)

    def __getitem__(self, idx):
        #if torch.is_tensor(idx):
         #   idx = idx.tolist()
       
        annotation = self.annotation[idx]
        path = annotation['data']['image']
        print(path)
        
        image = Image.open(path)
        
        image_2_npArray = np.asarray(image)
        image = torchvision.transforms.ToTensor()(image_2_npArray)
        
        result = annotation['annotations'][0]['result'][0]
        
        
        original_width = result['original_width']
        original_height = result['original_height']
        
        value = result['value']
        x = (value['x'] / 100.0) * original_width
        y = (value['y'] / 100.0) * original_height
        
        width = (value['width'] / 100.0) * original_width
        height = (value['height'] / 100.0) * original_height
        
        return image, int(x + (width / 2) ), int(y + (height / 2) ) 


In [3]:
def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='r')
    plt.pause(0.001)  # pause a bit so that plots are updated

In [4]:
planes_dataset = PlanesDataset(annotation_file='/home/alex/Рабочий стол/python/yandex/planes/labels.json',
                                    root_dir='/home/alex/Рабочий стол/python/yandex/planes/data/')


In [5]:
def show_image(img_tensor):
    image = np.moveaxis(img_tensor.numpy(), 0, 2)
    
    #==================================
    dpi = 80
    height, width, depth = image.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(image)

    plt.show()

In [13]:
try:
    img_tensor, x, y = planes_dataset[8]
    for i in range(0, img_tensor.size()[2]):
        img_tensor[:, y, i] = -1

    for i in range(0, img_tensor.size()[1]):
        img_tensor[:, i, x] = -1    
    
    show_image(img_tensor)
except ValueError:
    print("Oops!  That was no valid number.  Try again...")

/home/alex/Рабочий стол/python/yandex/planes/data/57.jpeg


FileNotFoundError: [Errno 2] No such file or directory: '/home/alex/Рабочий стол/python/yandex/planes/data/57.jpeg'

In [7]:
img_tensor.size()

torch.Size([3, 696, 1024])

In [8]:
y

371

In [9]:
x

505

In [10]:
type(x)

int